<a href="https://colab.research.google.com/github/minh-chaudang/IntroAI/blob/main/Bloxorz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import copy

In [2]:
# Position of the block in the map
class Position:
  def __init__(self, i1, j1, i2, j2, parent = None):
    self.i1 = i1
    self.j1 = j1
    self.i2 = i2
    self.j2 = j2
    self.parent = parent

  def print(self):
    print("(", self.i1, self.j1, ")",  "(", self.i2, self.j2, ")")

  def get_state(self):
    if self.i1 == self.i2 and self.j1 == self.j2: return "STANDING"
    elif (self.j1 == self.j2+1 or self.j1 == self.j2-1) and self.i1 == self.i2: return "LYING_HORIZONTALLY"
    elif (self.i1 == self.i2+1 or self.i1 == self.i2-1) and self.j1 == self.j2: return "LYING_VERTICALLY"
    else: return "SPLIT"
  
  def move(self, instruction):
    state = self.get_state()
    if instruction == "UP": 
      if state == "STANDING": return Position(self.i1-2, self.j1, self.i2-1, self.j2, self)
      elif state == "LYING_HORIZONTALLY": return Position(self.i1-1, self.j1, self.i2-1, self.j2, self)
      elif state == "LYING_VERTICALLY": return Position(min(self.i1, self.i2)-1, self.j1, min(self.i1, self.i2)-1, self.j2, self)
    elif instruction == "DOWN": 
      if state == "STANDING": return Position(self.i1+2, self.j1, self.i2+1, self.j2, self)
      elif state == "LYING_HORIZONTALLY": return Position(self.i1+1, self.j1, self.i2+1, self.j2, self)
      elif state == "LYING_VERTICALLY": return Position(max(self.i1, self.i2)+1, self.j1, max(self.i1, self.i2)+1, self.j2, self)
    elif instruction == "LEFT":
      if state == "STANDING": return Position(self.i1, self.j1-2, self.i2, self.j2-1, self)
      elif state == "LYING_HORIZONTALLY": return Position(self.i1, min(self.j1, self.j2)-1, self.i2, min(self.j1, self.j2)-1, self)
      elif state == "LYING_VERTICALLY": return Position(self.i1, self.j1-1, self.i2, self.j2-1, self)
    elif instruction == "RIGHT":
      if state == "STANDING": return Position(self.i1, self.j1+2, self.i2, self.j2+1, self)
      elif state == "LYING_HORIZONTALLY": return Position(self.i1, max(self.j1, self.j2)+1, self.i2, max(self.j1, self.j2)+1, self)
      elif state == "LYING_VERTICALLY": return Position(self.i1, self.j1+1, self.i2, self.j2+1, self)
    else: return "INVALID INPUT"

  def expand(self):
    return [self.move("UP"), self.move("DOWN"), self.move("LEFT"), self.move("RIGHT")]
  

In [19]:
class BLOXORZ:
  def __init__(self, map, initial_position):
    self.map = map
    self.initial_position = initial_position

  def is_goal(self, current_position):
    return current_position.get_state() == "STANDING" and self.map[current_position.i1][current_position.j1] == 10

  def is_valid_position(self, current_position):
    if current_position.i1 < 0 or current_position.i2 < 0 or current_position.j1 < 0 or current_position.j2 < 0: return False
    if current_position.i1 >= len(self.map) or current_position.i2 >= len(self.map) or current_position.j1 >= len(self.map[0]) or current_position.j2 >= len(self.map[0]): return False

    if current_position.get_state() == "STANDING": return self.map[current_position.i1][current_position.j1] > 1
    else: return self.map[current_position.i1][current_position.j1] > 0 and self.map[current_position.i2][current_position.j2] > 0
  
  def DFS(self, stack, visited, loop):
    while len(stack) > 0 and (([stack[-1].i1, stack[-1].j1, stack[-1].i2, stack[-1].j2] in visited or [stack[-1].i2, stack[-1].j2, stack[-1].i1, stack[-1].j1] in visited or not self.is_valid_position(stack[-1]))):
      stack.pop()

    if len(stack) == 0: return "No solutions"
    else:
      if self.is_goal(stack[-1]): 
        print("Goal reached after", loop, "loops")
        path = [stack[-1]]
        while path[-1].parent != None: path.append(path[-1].parent)
        path.reverse()
        return path
      else:
        this_position = stack[-1]
        # this_position.print()
        visited.append([this_position.i1, this_position.j1, this_position.i2, this_position.j2])
        stack.pop()
        stack += this_position.expand()
        return self.DFS(stack, visited, loop+1)

In [4]:
pos = Position(3,4,3,4)
children = pos.expand()
for child in children: child.print()

( 1 4 ) ( 2 4 )
( 5 4 ) ( 4 4 )
( 3 2 ) ( 3 3 )
( 3 6 ) ( 3 5 )


In [20]:
initBlock = Position(3,1,3,1)
map = [[0,0,0,0,0,0,2,2,2,2,2,2,2,0,0],
        [2,2,2,2,0,0,2,2,2,0,0,2,2,0,0],
        [2,2,2,2,2,2,2,2,2,0,0,2,2,2,2],
        [2,2,2,2,0,0,0,0,0,0,0,2,2,10,2],
        [2,2,2,2,0,0,0,0,0,0,0,2,2,2,2],
        [0,0,0,0,0,0,0,0,0,0,0,0,2,2,2]]
game = BLOXORZ(map, initBlock)
path = game.DFS([initBlock], [], 0)
for position in path: position.print()


Goal reached after 64 loops
( 3 1 ) ( 3 1 )
( 3 3 ) ( 3 2 )
( 4 3 ) ( 4 2 )
( 4 1 ) ( 4 1 )
( 2 1 ) ( 3 1 )
( 2 2 ) ( 3 2 )
( 2 3 ) ( 3 3 )
( 4 3 ) ( 4 3 )
( 4 1 ) ( 4 2 )
( 4 0 ) ( 4 0 )
( 2 0 ) ( 3 0 )
( 1 0 ) ( 1 0 )
( 1 2 ) ( 1 1 )
( 2 2 ) ( 2 1 )
( 2 3 ) ( 2 3 )
( 2 5 ) ( 2 4 )
( 2 6 ) ( 2 6 )
( 2 8 ) ( 2 7 )
( 1 8 ) ( 1 7 )
( 0 8 ) ( 0 7 )
( 0 9 ) ( 0 9 )
( 0 11 ) ( 0 10 )
( 0 12 ) ( 0 12 )
( 2 12 ) ( 1 12 )
( 2 11 ) ( 1 11 )
( 3 11 ) ( 3 11 )
( 3 13 ) ( 3 12 )
( 3 14 ) ( 3 14 )
( 5 14 ) ( 4 14 )
( 5 13 ) ( 4 13 )
( 5 12 ) ( 4 12 )
( 3 12 ) ( 3 12 )
( 3 14 ) ( 3 13 )
( 4 14 ) ( 4 13 )
( 4 12 ) ( 4 12 )
( 2 12 ) ( 3 12 )
( 2 13 ) ( 3 13 )
( 2 14 ) ( 3 14 )
( 4 14 ) ( 4 14 )
( 4 12 ) ( 4 13 )
( 5 12 ) ( 5 13 )
( 5 14 ) ( 5 14 )
( 3 14 ) ( 4 14 )
( 3 13 ) ( 4 13 )
( 2 13 ) ( 2 13 )
( 2 11 ) ( 2 12 )
( 3 11 ) ( 3 12 )
( 3 13 ) ( 3 13 )


In [ ]:
game.DFS([initBlock], [])

Popped


'No solutions'